<a href="https://colab.research.google.com/github/dhitology/pdf/blob/master/008_Keyword_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Muhammad Apriandito Arya Saputra - 2020*


---



# Keyword Network

## Mengambil Data dari Github

In [0]:
# Clone Library and Data from Github
! git clone https://github.com/dhitology/pdf

In [0]:
# Current Working Directory
import os
os.getcwd()

In [0]:
# Set Working Directory
os.chdir('pdf/journal/sage')

In [0]:
# Lokasi working directory saat ini
wd = os.getcwd()
print(wd)

## Mengambil Informasi Author dari Beberapa Dokumen PDF dengan PyPDF2

In [0]:
# Install Library
! pip install PyPDF2

In [0]:
# Import Library
import PyPDF2

In [0]:
# Mengambil Nama Dokumen PDF Dalam direktori
pdfname = []
for filename in os.listdir("."):
  if filename.endswith(".pdf"):
    pdfname.append(filename)

# Print 
print(pdfname)

In [0]:
# Membuat list kosong
keyword =[]

In [0]:
# Looping untuk mengambil penulis dari setiap dokumen
for filename in pdfname:
  pdfFileObj = open(filename,"rb")
  journal = PyPDF2.PdfFileReader(pdfFileObj)
  journal_info =  journal.getDocumentInfo()
  keyword.append(journal_info['/Keywords'])

In [0]:
# Print Keyword
print(keyword)

## Pre-Processing

In [0]:
# Import Re
import re

# Hapus kata oleh dengan fungsi re.sub()
daftar_keyword = [re.sub(r",", " ", i) for i in keyword]
daftar_keyword = [re.sub(r";", " ", i) for i in daftar_keyword]

In [0]:
daftar_keyword

In [0]:
# Import Pandas
import pandas as pd

# Convert list diatas menjadi dataframe dengan nama df_team lalu print
df_keyword = pd.DataFrame(daftar_keyword, columns= ['text'])
df_keyword

In [0]:
# Select Text
text = df_keyword['text']
type(text)

In [0]:
# Convert ke lower Case
text_lower = text.str.lower()

In [0]:
import nltk
from nltk import bigrams
from nltk.tokenize import word_tokenize
from random import seed

nltk.download('punkt')

In [0]:
# Tokenize
text_data = [word_tokenize(i) for i in text_lower]
print(text_data)

## Membuat Text Matrix

In [0]:
# Fungsi untuk membuat Co-occureness
def generate_co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    vocab_index = {word: i for i, word in enumerate(vocab)}
 
    # Create bigrams from all words in corpus
    bi_grams = list(bigrams(corpus))
 
    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))
 
    # Initialise co-occurrence matrix
    # co_occurrence_matrix[current][previous]
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))
 
    # Loop through the bigrams taking the current and previous word,
    # and the number of occurrences of the bigram.
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]
        pos_current = vocab_index[current]
        pos_previous = vocab_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count
    co_occurrence_matrix = np.matrix(co_occurrence_matrix)
 
    # return the matrix and the index
    return co_occurrence_matrix, vocab_index

In [0]:
# Create one list using many lists
import itertools
import numpy as np

data = list(itertools.chain.from_iterable(text_data))
matrix, vocab_index = generate_co_occurrence_matrix(data)

data_matrix = pd.DataFrame(matrix, index=vocab_index,
                             columns=vocab_index)

In [0]:
# Menampilkan 5 Baris pertama matrix
data_matrix.head()

In [0]:
# Save matrix as CSV, so we can analyze it Gephi
data_matrix.to_csv('matrix.csv', encoding='utf-8')

## Visualisasi Network

In [0]:
# Import NetworkX
import networkx as nx
G = nx.from_pandas_adjacency(data_matrix)

In [0]:
# Convert ke Edgelist
edgelist = nx.to_pandas_edgelist(G)
edgelist.head()

In [0]:
# Visualisasi Network
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
nx.draw(G, with_labels=True, 
        node_color='skyblue', node_size=1200, 
        arrowstyle='->',arrowsize=20, edge_color='r',
        font_size=9,
        pos=nx.kamada_kawai_layout(G))